In [ ]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
# pip install python-dotenv

In [2]:
import minsearch
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

# docs_raw[0]


In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

# documents[0]

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q='the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5)


In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [ ]:
import os

# Iterate through all environment variables and print them
for key, value in os.environ.items():
    print(f"{key}: {value}")


In [11]:
from openai import OpenAI

In [12]:
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [15]:
response.choices[0].message.content

"Whether or not you can still enroll in a course that has already started typically depends on the policies of the educational institution offering the course. Here are a few steps you can take to find out:\n\n1. **Contact the Instructor or Course Administrator**: Often, the instructor will be able to tell you if late enrollment is possible and what the requirements might be.\n\n2. **Check the Academic Calendar**: Some institutions have a cutoff date for adding classes, which is usually detailed in the academic calendar or the institution's website.\n\n3. **Consult the Registrar’s Office**: The registrar or admissions office can provide information on late enrollment and any associated penalties or requirements.\n\n4. **Online Platforms**: If the course is offered via an online platform, check the platform’s FAQ or help section for policies regarding late enrollment.\n\n5. **Approval Process**: Sometimes, you might need to get special approval from a department head or dean.\n\n6. **Ca

In [20]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.+
IF CONETXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [22]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [28]:
promt = prompt_template.format(question=q, context=context).strip()

In [31]:
print(promt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.+
IF CONETXT does not contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fina

In [32]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": promt}]
)

In [33]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."